<a href="https://colab.research.google.com/github/hyeyoungkoh/Data-driven-hollow-section-column-design/blob/main/ML/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Version 1.0, March 2023*
<br>
By Hyeyoung Koh hyeyoung.koh@wisc.edu and Hannah Blum hannah.blum@wisc.edu
<br>
<font color=grey>Civil and Environmental Engineering, University of Wisconsin-Madison </font>

1. Import Python packages needed

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

2. Create a dataframe

In [3]:
data_url = 'https://raw.githubusercontent.com/HyeyoungKoh/Data-driven-steel-design/main/dataset.csv'
data = pd.read_csv(data_url, header=0)

data = data[data!= -1] # remove Nu=-1 (publish data without -1)
data = data.dropna()

data_x = data[['Forming','$H$', '$B$', '$r_o$', '$t$', '$L_{c}$', '$F_y$']] #primary parameters
# data_x = data[['Forming','$A_{e}$', '$I$', '$L_{c}/r$','$F_y$']] #secondary parameters

data['Forming'].replace(['Cold-formed', 'Hot-rolled'],[0, 1], inplace=True) #replace labels of 'Cold-formed' and 'Hot-rolled' to numeric values to make it machine-readable


3. Split the data into train and test sets

In [4]:
y = pd.DataFrame(data,columns=['$N_u (kN)$']) #response variable, Nu
X= pd.DataFrame(data,columns=data_x.columns) #input variables

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

#Scale the data
sc_X = StandardScaler()
sc_y = StandardScaler()
X_sc = sc_X.fit_transform(X)
y_sc = sc_y.fit_transform(y)

X_train_sc, X_test_sc, y_train_sc, y_test_sc = train_test_split(X_sc, y_sc, test_size=0.3, random_state=123)


4. Train the MLP model

In [ ]:
mlp= MLPRegressor(random_state=123, activation='relu', alpha=5e-05, hidden_layer_sizes=(100,150,50),learning_rate='constant', solver= 'lbfgs',max_iter=1000) #primary parameters
# mlp= MLPRegressor(random_state=123, activation='relu', alpha=5e-05, hidden_layer_sizes=(100,150),learning_rate='constant', solver= 'lbfgs',max_iter=1000) #secondary parameters

mlp.fit(X_train_sc, y_train_sc.ravel())

# y_pred_mlp_train = mlp.predict(X_train_sc) #prediction of the training set
# y_pred_mlp = mlp.predict(X_test_sc) #prediction of the test set

# y_pred_mlp_de_train = mlp.predict(X_train_sc)*np.std(y).item()+np.mean(y).item() #De-normalize prediction of the training set
# y_pred_mlp_de = y_pred_mlp*np.std(y).item()+np.mean(y).item() #De-normalize prediction of the test set

# r2_train = r2_score(y_train, y_pred_mlp_de_train)
# r2_test = r2_score(y_test, y_pred_mlp_de)

# print('r2 score for training = {:8.4f}'.format(r2_train))
# print('r2 score for testing = {:8.4f}'.format(r2_test))

5. Get buckling strength of hollow section columns using the input parameters

In [8]:
Forming = 0 # use 0 for cold-formed steel and 1 for hot-rolled steel
H = 150 # overall height, mm
B = 150 # overall width, mm
ro = 9 # outer corner radius, mm
t = 5 # thickness, mm
Lc = 1700 # the effective length, mm
Fy = 600 # yield stress, MPa

#scale the parameters
Forming_s = (Forming-np.mean(X['Forming']))/np.std(X['Forming'])
H_s = (H-np.mean(X['$H$']))/np.std(X['$H$'])
B_s = (B-np.mean(X['$B$']))/np.std(X['$B$'])
ro_s = (ro-np.mean(X['$r_o$']))/np.std(X['$r_o$'])
t_s = (t-np.mean(X['$t$']))/np.std(X['$t$'])
Lc_s = (Lc-np.mean(X['$L_{c}$']))/np.std(X['$L_{c}$'])
Fy_s = (Fy-np.mean(X['$F_y$']))/np.std(X['$F_y$'])

input = pd.DataFrame([[Forming_s, H_s, B_s, ro_s, t_s, Lc_s, Fy_s]], columns=data_x.columns)
y_pred = mlp.predict(input) #get prediction using the scaled parameters
y_pred_de = y_pred*np.std(y).item()+np.mean(y).item() #De-normalize y_pred

print('Buckling strength = {:6.1f} kN'.format(y_pred_de[0]))

Buckling strength = 1493.3 kN


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
